In [ ]:
# Setting up a custom stylesheet in IJulia
file = open("style.css") # A .css file in the same folder as this notebook file
styl = read(file,String) # Read the file
HTML("$styl") # Output as HTML

<h1> "for" loops and date-time formats: converting string values into usable time data </h1>

<h2>In this lecture</h2>

- [Outcome](#Outcome)
- [Converting a date string to DateTime format](#Converting-a-date-string-to-DateTime-format)
- [``for`` loops](#"for"-loops)
- [Converting column 1 DateTime type](#Converting-column-1-DateTime-type)
- [Creating data giving time in days since 22 March 2014](#Creating-data-giving-time-in-days-since-22-March-2014)
- [Exporting the converted data](#Exporting-the-converted-data)

[Back to the top](#In-this-lecture)

<h2>Outcome</h2>

After this lecture, you will be able to
- Convert a string with date data to Julia's DateTime format
- Use ``Dates.datetime2rata()`` to calculate the number of days between two DateTime dates
- Use a ``for`` loop to convert the dates of the epidemic data to days since 22 March 2014
- Use ``writedlm()`` to save the converted data as .csv file

[Back to the top](#In-this-lecture)

We start by reading in the data:

In [ ]:
using DelimitedFiles
wikiEVDraw = DelimitedFiles.readdlm("wikipediaEVDraw.csv", ',')  # 

[Back to the top](#In-this-lecture)

<h2>Converting a date string to DateTime format</h2>

Remarkably enough, data on dates and times are among the fiddliest things a data scientist has to deal with. There are a huge number of different ways in which such data are reported, and moreover there are conflicting standards of how to deal irregularities (month lengths aren't all the same, some years are leap years, time zones shift ...).

In consequence, every computer language that deals with date-time data has a rich array of  functions to deal with it. In Julia, they are in a package called Dates. Of this package, we will use the functions ``DateTime()`` and ``Dates.datetime2rata()``.

Why does one of them use the dot syntax and the other does not? The answer is that when you start up Julia, only a few of the functions in the package Dates are visible. These functions include ``DateTime()`` but not ``datetime2rata()``. However, we are able to access the other functions via the dot notation. We will talk more about packages in the next lecture.

The ``DateTime()`` function uses a format string convert string data such as we see in column one into Julia DateTime data.

A format string is something one sees in many computation contexts. Here, it tells Julia in what form to expect the data. Looking at the date strings in the data, they have a number for the day, then space, then an abbreviation for the month, then a space, then a number for the year. The appropriate format string is therefore ``"d u y"``. These formats have limitations: ``d`` accepts one- and two-digit days (which should always work) and ``y`` accepts two- and four-digit years (which should mostly work), but ``u`` accepts only three-letter abbreviations. Unfortunately, data where the month names otherwise abbreviated are fairly common and they will need a different format string.

Here is an example of how the conversion works


In [ ]:
using Dates
Dates.DateTime(wikiEVDraw[1,1], "d u y")

[Back to the top](#In-this-lecture)

<h2>"for" loops</h2>

Now we need to do this conversion for every element in column 1 of the matrix. The way to do this is with a ``for`` loop.

``for`` loops are extremely important in computing, and in Julia even more so. This is because many items that are vectorised in languages like Matlab and Python are explicitly computed in ``for`` loops in Julia. It may surprise many of you who know about speeding up computations using vectorisation, but it is frequently the case that a loop in Julia runs *faster* than the equivalent vectorised code.

``for`` loops have a simple structure: the outside is the ``for ... end`` part and the inside is a code block executed repeatedly. Exactly how many times is determined by the the ``for ... end`` part.

In the two examples below, we use ``println()`` to show the value of the variable over which the ``for`` loop runs. Notice that these values do not have to be a sequence of integers.


In [ ]:
for num = 3:7    # here, the colon is used to specify a range; we will see this again
    println("num is now $num")       # remember that the special character `$` is used for string interpolation
end

testvalues = [23, "my name is not a name", 'ℵ']      # an array with some rather odd elements
for x in testvalues    # a for loop can iterate over an array
        println("The value of x is now $x")
    end

It is important to get the first line of a ``for`` loop exactly right. It has the structure 

"variable = iterable"

Here, "iterable" is anything that is arranged in a sequence. Not all types are, but they certainly include ranges (created with the colon operator ``:``) and any single dimension of an array. The ``=`` is an assignment operator, and it assigns to "variable" the values in "iterable", one after the other. That is, during each pass through the loop, "variable" has the value of exactly one of the items in "iterable". 

[Back to the top](#In-this-lecture)

<h2>Converting column 1 DateTime type</h2>

Now we use a ``for`` loop twice. Firstly we create a one-dimensional array containing just column one---it uses array slicing, for conversion to values with DateTime type.


In [ ]:
col1 = wikiEVDraw[:, 1]  # the colon means all the data in the column, the 1 means the first column

We use a ``for`` loop to overwrite the data in the variable ``col1`` with converted data using ``DateTime()`` as follows:

In [ ]:
for i = 1:length(col1)
    col1[i] = Dates.DateTime(col1[i], "d u y")  # note that this replaces the previous value in col1[i]
end

In [ ]:
col1  # let's view it!

<h2>Creating data giving time in days since 22 March 2014</h2>


Finally, we create the variable "epidays". This calls to mind the concept of *epidemic day*, which is simply a way to indicate how long an epidemic has been running. We will assume that the epidemic started on 22 March 2014, with a total of 49 cases, because that is the  first date for which we have data.


Note that this is in keeping with the spirit of modelling: we are trying to do the best we can with the data we have. Even when we know that the epidemic has been traced back to a single case in early December 2013, that information is not in the table of data before us. We should not forget about it, but neither should we attempt to include it in the data.

The function we use is ``Dates.datetime2rata()``. The "Rata Die days" format is a specialised date format we will not discuss here (see https://en.wikipedia.org/wiki/Rata_Die for information). The important thing is that this function, applied to a given date, gives the number of days since 1 January of the year 0001. As follows:


In [ ]:
Dates.datetime2rata(col1[1])

<h2>Exporting the converted data</h2>


We create a function to express the number of days since epidemic day zero, which is the value of ``col1[54]`` which is of course 22 March 2014.

Then we iterate that function with a for loop over all the elements in col1 to create epidays. Note that the variable epidays is created before the start of the loop. This is, in general, good practice: if you know what array you want to fill, then initialise that array before you start filling it.

In [ ]:
dayssincemar22(x) = Dates.datetime2rata(x) - Dates.datetime2rata(col1[54])
epidays = Array{Int64}(undef,54)
for i = 1:length(col1)
    epidays[i] = dayssincemar22(col1[i])
end


Finally, we overwite column 1 of our data array with ``epidays``, and save it using ``writedlm()``. It is a good idea to use a new filename, so that all the work that went into extracting the data from wikipedia is not lost. You never know when you might want the original dates again!

In [ ]:
wikiEVDraw[:, 1] = epidays
DelimitedFiles.writedlm("wikipediaEVDdatesconverted.csv", wikiEVDraw, ',')  
#         note the delimiter ... the Julia default is a tab; to get .csv, we must specify the comma

[Back to the top](#In-this-lecture)